In [1]:
import mlflow
import pandas as pd
from openai import OpenAI
import os
from wandb.sdk.data_types.trace_tree import Trace
import wandb
import configparser

In [20]:
config = configparser.ConfigParser()
# Read the configuration file
config.read('config.ini')
api_key_openai = config.get('credentials', 'api_key_openai')
api_key_mistral = config.get('credentials', 'api_key_mistral')
surfdrive_url_input_sentences = config.get('credentials', 'surfdrive_url_input_sentences')
surfdrive_url_prompts = config.get('credentials', 'surfdrive_url_prompts')

### Input Prompts

In [21]:
neutral_sentences = pd.read_csv(surfdrive_url_input_sentences,sep=';')['sentences']
neutral_sentences

0     I’m all about that food. I usually kick off th...
1     Just getting my vitamins in at the school cant...
2     De Pizzabakkers sell this vegan pizza with che...
3     I entered the world of vegan foods lately. Nex...
4     This vegan fried chicken from KFC is on the sp...
5     Just having this vegan hotdog from the school ...
6     Just had my first vegan cake at "groene bakker...
7     This vegan chocolate is on point. Its with oat...
8     I'm all about fast food. I'm having some fries...
9     I went to the "Groene Burger" fast food restau...
10    I am at the food feestje in "Koningsplein". I ...
11    I just ate a vegan schnitzel. The flavors and ...
12    The school canteen has vegan Buddha bowls late...
13    I am all over this Italian restaurant. I had s...
14    I am having some new food: vegan Magnum. Just ...
15    I'm going through the city. I am going for som...
16    I am having a midday snack later outside. Soy ...
17    I am all about hot chocolate and some bake

In [44]:
df_prompts = pd.read_csv(surfdrive_url_prompts,sep=';').reset_index()
type(df_prompts)

pandas.core.frame.DataFrame

In [67]:
prompt_string = df_prompts['promptContent'][0]
replacement_string = "your_replacement_string"

prompt_string = f"{prompt_string.replace('{}', f'{{{replacement_string}}}')}"

print(prompt_string)

Here is some text {your_replacement_string}. Here is a rewrite of the text, which is more positive { 


### Mixtral Setup

In [3]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

model = "mistral-medium"
final_output = []
client = MistralClient(api_key=os.environ["MISTRAL_API_KEY"])


num_prompts = df_prompt.shape[0]

for i in range(0,num_prompts-1):
    query = df_prompt['rewritten'][i]
    original = df_prompt['original'][i]
    messages = [
        ChatMessage(role = "user", content = query)
    ]
    
    # No streaming
    chat_response = client.chat(
        model=model,
        messages=messages,
    )
    
    final_output.append({'original': original,'output': chat_response.choices[0].message.content,"model": chat_response.model, "prompt_tokens" : chat_response.usage.prompt_tokens,"completion_tokens" : chat_response.usage.completion_tokens,"object" : chat_response.object})

df_mistral_output = pd.DataFrame(final_output)

0
1
2
3
4
5
6
7


In [4]:
wandb.init(project="lmm-evaluate", name="mistral-medium-run")
# log df as a table to W&B for interactive exploration
wandb.log({"free_style_transfer_mistral_medium_": wandb.Table(dataframe=df_mistral_output)})

df_mistral_output.to_csv("df_mistral_medium_output.csv")

# log csv file as an dataset artifact to W&B for later use

artifact = wandb.Artifact("df_mistral_medium_output", type="dataset")
artifact.add_file("df_mistral_medium_output.csv")
wandb.log_artifact(artifact)

wandb.finish()

2024-01-29 16:29:30,820 ERROR wandb.jupyter: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bojan-2110. Use `wandb login --relogin` to force relogin


### gpt-3.5-turbo Setup

In [27]:
# client = OpenAI()
# system_msg = "You are an expert in text style transfer."
# temperature=0.2
# max_tokens=256
# frequency_penalty=0.0

# final_output = []
# num_prompts = df_prompt.shape[0]

# query = df_prompt['rewritten'][1]
# original = df_prompt['original'][1]
# message=[{"role": "system", "content": system_msg}, {"role": "user", "content":query}]
# chat_response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages = message,
#     temperature=temperature,
#     max_tokens=max_tokens,
#     frequency_penalty=frequency_penalty
# )

# print(chat_response)

In [18]:
client = OpenAI()
system_msg = "You are an expert in text style transfer."
temperature=0.2
max_tokens=256
frequency_penalty=0.0

final_output = []
num_prompts = df_prompt.shape[0]

for i in range(0,num_prompts-1):
    query = df_prompt['rewritten'][i]
    original = df_prompt['original'][i]
    message=[{"role": "system", "content": system_msg}, {"role": "user", "content":query}]
    chat_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = message,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )
    final_output.append({'original': original,'output': chat_response.choices[0].message.content,"model": chat_response.model, "prompt_tokens" : chat_response.usage.prompt_tokens,"completion_tokens" : chat_response.usage.completion_tokens,"object" : chat_response.object, "temperature": temperature})

df_gpt_35_output = pd.DataFrame(final_output)

Here is some text: {Embrace the power of plant-based nutrition for a healthier you!}. Here is a rewrite of the text, which is more positive: {
Here is some text: {Discover the joy of eating fresh fruits and vegetables every day.}. Here is a rewrite of the text, which is more positive: {
Here is some text: {Going green with your diet can lead to a lighter, brighter you!}. Here is a rewrite of the text, which is more positive: {
Here is some text: {Explore the world of tasty and nutritious meat-free meals.}. Here is a rewrite of the text, which is more positive: {
Here is some text: {Feel energized and revitalized with whole, plant-based foods.}. Here is a rewrite of the text, which is more positive: {
Here is some text: {Make every meal an adventure with colorful, vegan dishes.}. Here is a rewrite of the text, which is more positive: {
Here is some text: {Nourish your body and soul with nature’s best offerings.}. Here is a rewrite of the text, which is more positive: {
Here is some text

In [21]:
wandb.init(project="lmm-evaluate", name="gpt-3-5-run")
# log df as a table to W&B for interactive exploration
wandb.log({"free_style_transfer_gpt_3_5_": wandb.Table(dataframe=df_gpt_35_output)})

df_gpt_35_output.to_csv("df_gpt_3_5_output.csv")

# log csv file as an dataset artifact to W&B for later use

artifact = wandb.Artifact("df_gpt_3_5_output", type="dataset")
artifact.add_file("df_gpt_3_5_output.csv")
wandb.log_artifact(artifact)

wandb.finish()

### gpt 4 setup

In [22]:
client = OpenAI()
system_msg = "You are an expert in text style transfer."
temperature = 0.2
max_tokens = 256
frequency_penalty = 0.0

final_output = []
num_prompts = df_prompt.shape[0]
for i in range(0,num_prompts-1):
    query = df_prompt['rewritten'][i]
    original = df_prompt['original'][i]
    message=[{"role": "system", "content": system_msg}, {"role": "user", "content":query}]
    chat_response = client.chat.completions.create(
        model="gpt-4",
        messages = message,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )
    final_output.append({'original': original,'output': chat_response.choices[0].message.content,"model": chat_response.model, "prompt_tokens" : chat_response.usage.prompt_tokens,"completion_tokens" : chat_response.usage.completion_tokens,"object" : chat_response.object, "temperature": temperature})

df_gpt_4_output = pd.DataFrame(final_output)

In [24]:
wandb.init(project="lmm-evaluate", name="gpt-4-run")
# log df as a table to W&B for interactive exploration
wandb.log({"free_style_transfer_gpt_4_": wandb.Table(dataframe=df_gpt_4_output)})

df_gpt_4_output.to_csv("df_gpt_4_output.csv")

# log csv file as an dataset artifact to W&B for later use

artifact = wandb.Artifact("df_gpt_4_output", type="dataset")
artifact.add_file("df_gpt_4_output.csv")
wandb.log_artifact(artifact)

wandb.finish()